# Binary Output in the LSTM

In [4]:
import matplotlib.pyplot as plt
import mlflow
import mlflow.keras
import numpy as np
from numpy import ndarray
import pandas as pd
import sys

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow as tf

%load_ext lab_black

sys.path.append("/Users/renero/Documents/SideProjects/trader/src")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/predictor")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/utils")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/indicators")


from cs_dictionary import CSDictionary
from lstm import lstm_1layer, lstm_2layer
from sequences import sequences
from ticks import Ticks
from metrics import metrics

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [5]:
argv = [
    "",
    "-c",
    "../src/predictor/params.dax.yaml",
    "-f",
    "../data/^GDAXI/dax_2018_2019.csv",
    "--window",
    "8",
    "--epochs",
    "400",
    "train",
]
params = CSDictionary(args=argv)

ticks = Ticks(params, params.input_file).scale()

2020-09-17 13:37:51 - INFO  - CSDictionary:__init__           - Using configuration parameters from: ../src/predictor/params.dax.yaml


In [6]:
ticks.append_indicator(["trend", "median_filter"])
ticks.data.head()

,open,high,low,close,trend,median_filter
Datetime,,,,,,
2018-01-02,0.71,0.66,0.61,0.67,1.0,0.67
2018-01-03,0.73,0.78,0.77,0.79,1.0,0.79
2018-01-04,0.90,0.99,0.96,1.00,1.0,1.00
2018-01-05,1.07,1.13,1.13,1.17,1.0,1.17
2018-01-08,1.26,1.21,1.26,1.22,1.0,1.17


The way it should be:

```python
    # to predict close and use OHLC values
>>> ticks.to_timewindows(predict='close')
>>> X, y, Xt, yt = ticks.split(test_size=0.1)
    
    # To generate X and y without test set
>>> ticks.to_timewindows(predict='close')
>>> X, y = ticks.split(test_size=0.0)

    # if we want to predict "trend"
>>> ticks.append_indicator('trend')
>>> ticks.to_timewindows(predict='trend')
>>> X, y , Xt, yt = split(test_size=0.1)
    
    # if we want to use less variables in prediction
>>> ticks.append_indicator(['moving_average', 'trend'])
>>> ticks.to_timewindows(predict='trend', train_columns=['close', 'moving_average'])
```

So, let's work on it.

In [12]:
X_train, y_train, X_test, y_test = ticks.to_timewindows(
    "trend", train_columns=["close", "trend"]
)

In [13]:
print(f"X_train.shape: {X_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_test.shape: {y_test.shape}")

X_train.shape: (424, 8, 2)
y_train.shape: (424,)
X_test.shape: (48, 8, 2)
y_test.shape: (48,)
